In [1]:
from tensorflow.contrib.keras.api.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation
import tensorflow as tf

/Users/studentuser/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
class TB(tf.keras.callbacks.TensorBoard):
    #  code posted by VladislavZavadskyy at https://github.com/keras-team/keras/issues/6692
    def __init__(self, log_every=1, **kwargs):
        super().__init__(**kwargs)
        self.log_every = log_every
        self.counter = 0

    def on_batch_end(self, batch, logs=None):
        self.counter += 1
        if self.counter % self.log_every == 0:
            for name, value in logs.items():
                if name in ['batch', 'size']:
                    continue
                summary = tf.Summary()
                summary_value = summary.value.add()
                summary_value.simple_value = value.item()
                summary_value.tag = name
                self.writer.add_summary(summary, self.counter)
            self.writer.flush()

        super().on_batch_end(batch, logs)

The code block below is where we define the folder we want to save the logs to, the number of epochs to train over, the number of items to train over before updating the weights, and whether or not you wish to save the trained model. 

In [4]:
LOGDIR = "/tmp/IntelAIworkshop/"
nbEpochs = int(1)
batchSize = int(32)
modelSave = True

Here we are installing/loading in the data. The x_train and x_test variables contain the image data for the training and validation files respectively. The y_train and y_test variables contain the associated labels for the data.

In [5]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

The training data contains pixel values between 0 and 255. Here we scale the data to values between 0 and 1. We do this primarily to slightly decrease training time. The same scaling is applied to both training and validation sets.

In [8]:
# Max scale all the image data
training_scaled = x_train / x_train.max()
test_scaled = x_test / x_test.max()

The labels are numerical values between 0 and 9. The images are sparsely coded, meaning each image can only be apart of a *single* category.

In [ ]:
# One hot encoding of labels
labels_train = tf.keras.utils.to_categorical(y_train, 10)
labels_test = tf.keras.utils.to_categorical(y_test, 10)

In [ ]:
# Define the model: 4 convolutional layers, 2 max pools
model = tf.keras.Sequential()

We can now begin adding layers onto the model. Our first layer after the input will be a 2-dimensional convolutional layer. 

We first define the number of feature maps that this convolutional layer will output, which will be 32. 

We then define the size of the receptive window that will pass over the input matrix starting from the top left and finishing at the bottom right. 

The **padding** arguement means that the output feature maps will be zero-padded to the same size of the input.

In Keras, we have to identify the size of the input in the first layer of the network. This is defined by the **input_shape** arguement. 

The **strides** arguement defines how far the receptive window moves and determines the overlap in your features. Our receptive window will be moving 1 pixel to the right and after it reaches the end of the row, it will move 1 pixel down. This essentially provides the maximal overlap in our features.

The activation function used in the convolutional layer can be defined inside or outside the layer call. Here we are using the *rectified linear* (relu) function. This is the best general purpose activation function, but is prone to creating "dead" nodes. If you get a lot of dead nodes, injecting *truncated normal* noise can revive them.

In [ ]:
with tf.variable_scope('conv1'):
    model.add(Conv2D(32, (3, 3), padding='same',
                     input_shape=training_scaled.shape[1:],
                     strides=(1, 1),
                     # kernel_initializer='TruncatedNormal',
                     name='conv1'))
    model.add(Activation('relu'))

In [ ]:
with tf.variable_scope('conv2'):
    model.add(Conv2D(32, (3, 3), padding='same',
                     strides=(1, 1),
                     # kernel_initializer='TruncatedNormal',
                     name='conv2'))
    model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), name='pool1'))
model.add(Dropout(0.25, name='dropout1'))

In [ ]:
with tf.variable_scope('conv3'):
    model.add(Conv2D(64, (3, 3), padding='same',
                     strides=(1, 1),
                     # kernel_initializer='TruncatedNormal',
                     name='conv3'))
    model.add(Activation('relu'))

In [ ]:
with tf.variable_scope('conv4'):
    model.add(Conv2D(64, (3, 3), padding='same',
                     strides=(1, 1),
                     # kernel_initializer='TruncatedNormal',
                     name='conv4'))
    model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), name='pool2'))

In [ ]:
model.add(Flatten(name='flat1'))  # converts feature maps to feature vectors

with tf.variable_scope('fc1'):
    model.add(Dense(1024,
                    # kernel_initializer='TruncatedNormal',
                    name='fc1'))
    model.add(Activation('relu'))

model.add(Dropout(0.5, name='dropout2'))  # reset half of the weights to zero

In [ ]:
with tf.variable_scope('output'):
    model.add(Dense(10,
                    name='output'))
    model.add(Activation('softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
tensorboard = TB(log_dir=LOGDIR,
                 histogram_freq=1,
                 batch_size=20,
                 write_graph=True,
                 write_grads=False,
                 write_images=False)

In [ ]:
print("Initializing the model...")
# fits the model on batches, waits to send the error back after the number of batches
model.fit(training_scaled,
          labels_train,
          batch_size=batchSize,
          epochs=nbEpochs,
          validation_data=(test_scaled, labels_test),
          verbose=1,
          callbacks=[tensorboard])

if modelSave:
    model.save('./cifarClassification.h5')

print('To run tensorboard, open up either Firefox or Chrome and type localhost:6006 in the address bar.')
print('Then run `tensorboard --logdir=%s` in your terminal.' % LOGDIR)
print('If youre on a Mac, provide the following flag: '
      '--host=localhost to the previous terminal string.')